In [1]:
from pymongo import MongoClient
from pathlib import Path
import os
client = MongoClient('mongodb://localhost:27017')


In [12]:
def fileName(filePath):
    return (os.path.splitext(os.path.split(maxFileName)[1]))[0]

def fileExt(filePath):
    return (os.path.splitext(os.path.split(maxFileName)[1]))[1]

In [23]:
p1 = os.listdir("v1.2.0/eval")
print(p1)
print(os.path.isdir(p1[0]))
print(os.path.isfile(p1[1]))
pathString = ("/"+str(p1[0]))
print(pathString)
patientDict = {}
fileNames = []
maxFileSize = 0 
maxFileName = ""
for root,dirs,files in os.walk("v1.2.0"):
    #print ("root: ",root)
    #print("\tdirs: ",dirs)
    #print("\t\t",str(files))
        
    if len(files)>0:
        for file in files:
            fileNames.append(str(root)+"/"+str(file))
            if os.path.getsize((str(root)+"/"+file)) > maxFileSize:
                maxFileSize = os.path.getsize((str(root)+"/"+file))
                maxFileName = (str(root)+"/"+file)
            else:
                continue
            
            
        #print ("\t\tFile: "+files[3])
        
print (maxFileName," : ",maxFileSize)
print (os.path.splitext(maxFileName))
print (os.path.split(maxFileName))
print (os.path.splitext(os.path.split(maxFileName)[1]))
print (fileName(maxFileName))
print (fileExt(maxFileName))

['03_tcp_ar_a', '02_tcp_le', '.DS_Store', '01_tcp_ar']
False
False
/03_tcp_ar_a
v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17/00001278_s002_t001.edf  :  67899648
('v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17/00001278_s002_t001', '.edf')
('v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17', '00001278_s002_t001.edf')
('00001278_s002_t001', '.edf')
00001278_s002_t001
.edf


In [24]:
print(fileNames[0:30])

['v1.2.0/.DS_Store', 'v1.2.0/eval/.DS_Store', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002.txt', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t002.lbl_bi', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t001.tse_bi', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t000.lbl_bi', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t003.tse_bi', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t002.lbl', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t002.tse', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t003.tse', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t003.lbl', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t001.lbl', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t001.tse', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t000.tse', 'v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_

In [16]:
directory = os.fsencode(directory_in_str)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".asm") or filename.endswith(".py"): 
        # print(os.path.join(directory, filename))
        continue
    else:
        continue

NameError: name 'directory_in_str' is not defined